In [1]:
import pandas as pd

In [2]:
df_soil_19 = pd.read_excel('2019_Metals_Results_-_Soil_Kitchen.xls')

In [3]:
df_soil_19.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,Sample,Street,Cross Street,Initials,Chemical,Lead,Arsenic,Copper,Iron,Manganese,Nickel,Zinc
1,NaN,NaN,NaN,NaN,NaN,Texas-specific Background,15,5.9,15,15000,300,10,30
2,NaN,NaN,NaN,NaN,NaN,GW-Soil-Ing,3,5,1000,NaN,1200,160,2400
3,NaN,NaN,NaN,NaN,NaN,Tot-Soil-Combined,500,24,1300,NaN,3900,840,9900
4,NaN,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13


In [4]:
df_soil_19 = df_soil_19.drop(columns=['Unnamed: 0'])

In [5]:
df_soil_19.columns = df_soil_19.iloc[0]

In [6]:
df_base_19 = df_soil_19.head()

In [7]:
df_soil_19 = df_soil_19.drop(index=[0,1,2,3,4])

In [8]:
df_soil_19 = df_soil_19[['Street', 'Cross Street', 'Lead', 'Arsenic']]

In [9]:
df_soil_18 = pd.read_csv('2015-18 Soil_Kitchen_Log.csv')

In [10]:
df_soil_18.head()

,Unique ID,Sample Number,Year,Street,Cross Street,"City,State",Full Address,Initials,Notes,Lead,...,Chromium,Arsenic,Copper,Mercury,Nickel,Selenium,Silver,Antimony,Cadimum,Iron
0,1-2015,1,2015,Adelphi,Amherst,"Austin, TX","Adelphi & Amherst,Austin, TX",MB,NaN,24,...,NaN,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3-2015,3,2015,Blessing,E. Anderson,"Austin, TX","Blessing & E. Anderson,Austin, TX",MB,NaN,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4-2015,4,2015,Cameron,E. Anderson,"Austin, TX","Cameron & E. Anderson,Austin, TX",MB,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5-2015,5,2015,Lake Austin,Mopac,"Austin, TX","Lake Austin & Mopac,Austin, TX",CW,NaN,27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6-2015,6,2015,Waterson,Mopac,"Austin, TX","Waterson & Mopac,Austin, TX",CW,NaN,190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_soil_18 = df_soil_18[['Street', 'Cross Street', 'Full Address', 'Lead', 'Arsenic']]

In [12]:
df_soil_18.head()

,Street,Cross Street,Full Address,Lead,Arsenic
0,Adelphi,Amherst,"Adelphi & Amherst,Austin, TX",24,NaN
1,Blessing,E. Anderson,"Blessing & E. Anderson,Austin, TX",15,NaN
2,Cameron,E. Anderson,"Cameron & E. Anderson,Austin, TX",NaN,NaN
3,Lake Austin,Mopac,"Lake Austin & Mopac,Austin, TX",27,NaN
4,Waterson,Mopac,"Waterson & Mopac,Austin, TX",190,NaN


In [13]:
df_loc_18 = pd.read_csv('Locations Key 2015-2017.csv')

In [14]:
df_loc_18.head()

,Address,Latitude,Longitude
0,"Hearwood,Thistlewood,Austin, TX",30.213898,-97.774223
1,"Silk Oak,Bradner,Austin, TX",30.185817,-97.839934
2,"Tahoe Trail,Jones Rd,Austin, TX",30.224777,-97.804105
3,"North Loop,Avenue H,Austin, TX",30.314667,-97.720800
4,"Easy Wind,Cannoneer,Austin, TX",30.344105,-97.717867


In [15]:
df_soil_18['Full Address'] = [street + ',' + cross_street + ', Austin, TX' for street, cross_street in zip(df_soil_18['Street'], df_soil_18['Cross Street'])]


In [16]:
df_soil_18.head()

,Street,Cross Street,Full Address,Lead,Arsenic
0,Adelphi,Amherst,"Adelphi , Amherst, Austin, TX",24,NaN
1,Blessing,E. Anderson,"Blessing , E. Anderson, Austin, TX",15,NaN
2,Cameron,E. Anderson,"Cameron , E. Anderson, Austin, TX",NaN,NaN
3,Lake Austin,Mopac,"Lake Austin , Mopac, Austin, TX",27,NaN
4,Waterson,Mopac,"Waterson , Mopac, Austin, TX",190,NaN


In [17]:
df_soil_19['Full Address'] = [str(street) + ',' + str(cross_street) + ', Austin, TX' for street, cross_street in zip(df_soil_19['Street'], df_soil_19['Cross Street'])]


In [18]:
df_soil_19.head()

,Street,Cross Street,Lead,Arsenic,Full Address
5,Travis,Belmal,341.32,0,"Travis,Belmal, Austin, TX"
6,Salem Walk Dr,Cedar Glen,8.61,0,"Salem Walk Dr,Cedar Glen, Austin, TX"
7,Byton Loop,Cherry Creek,0,5.28,"Byton Loop,Cherry Creek, Austin, TX"
8,Goodwind,Gunter,10.33,0,"Goodwind,Gunter, Austin, TX"
9,Sunshine Dr,51st,41.15,0,"Sunshine Dr,51st, Austin, TX"


In [19]:
df_soil_18[['Lead', 'Arsenic']] = df_soil_18[['Lead', 'Arsenic']].fillna(0)

In [37]:
df_loc_18_mean

,Full Address,Latitude,Longitude
0,"10th&Menden,Austin,TX",30.278192,-97.758641
1,"10th&Prospect--Back,Austin,TX",30.267153,-97.743061
2,"10th&Prospect--Front,Austin,TX",30.267153,-97.743061
3,"10thSt.,TheresaAve,Austin,TX",30.279738,-97.766031
4,"12th&Springdale,Austin,TX",30.278961,-97.687041
...,...,...,...
663,"Woodmoor,Tannehill,Austin,TX",30.239108,-97.787008
664,"Woodrow&Alegria,Austin,TX",30.334284,-97.730361
665,"Woodrow,Brentwood,Austin,TX",30.332230,-97.730884
666,"Yaupon,Dianella,Austin,TX",30.410022,-97.776868


In [21]:
df_loc_18 = df_loc_18.rename(columns={'Address': 'Full Address'})

In [22]:
df_loc_18['Full Address'] = df_loc_18['Full Address'].astype(str)
df_soil_18['Full Address'] = df_soil_18['Full Address'].astype(str)

In [23]:
df_loc_18['Full Address'] = df_loc_18['Full Address'].str.replace(pat=' ', repl='')

In [24]:
df_soil_18['Full Address'] = df_soil_18['Full Address'].str.replace(pat=' ', repl='')

In [25]:
df_loc_18_mean = df_loc_18.groupby(by='Full Address').mean().reset_index()

In [26]:
df_18 = pd.merge(df_soil_18, df_loc_18_mean, on='Full Address')

In [27]:
len(df_loc_18_mean)

668

In [28]:
len(df_soil_18)

781

In [29]:
len(df_18)

591

In [30]:
from geopy.geocoders import Nominatim

In [31]:
geolocator = Nominatim(user_agent="soil_analysis")
location = geolocator.geocode("175 5th Avenue NYC")
print(location.address)
# Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
# print((location.latitude, location.longitude))

Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan Community Board 5, Manhattan, New York County, New York, 10010, United States of America
